In [1]:
import sqlite3
import pandas as pd
db = './sqlite_db_pythonsqlite.db'

conn = sqlite3.connect(db)
c = conn.cursor()
c.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = []
print(c.fetchall())



[('Bookings',), ('Facilities',), ('Members',)]


In [34]:
#Q10


c.execute("""SELECT a.facility
          FROM (SELECT Facilities.name as facility, SUM(Bookings.slots * CASE WHEN Bookings.memid = 0 THEN Facilities.guestcost ELSE Facilities.membercost END) as revenue
            FROM Bookings
            FULL OUTER JOIN Facilities
            ON Facilities.facid
            GROUP BY Facilities.name) as a
          WHERE revenue < 1000 OR revenue IS NULL
          """)

print(c.fetchall())

[('Tennis Court 1',)]


In [39]:
#Q11

c.execute("""SELECT M.firstname||' '||M.surname as member, R.firstname||' '||R.surname as recommender
          FROM Members as M, Members as R
          WHERE M.recommendedby = R.memid
          """)

print(c.fetchall())

[('Janice Joplette', 'Darren Smith'), ('Gerald Butters', 'Darren Smith'), ('Nancy Dare', 'Janice Joplette'), ('Tim Boothe', 'Tim Rownam'), ('Ponder Stibbons', 'Burton Tracy'), ('Charles Owen', 'Darren Smith'), ('David Jones', 'Janice Joplette'), ('Anne Baker', 'Ponder Stibbons'), ('Jack Smith', 'Darren Smith'), ('Florence Bader', 'Ponder Stibbons'), ('Timothy Baker', 'Jemima Farrell'), ('David Pinker', 'Jemima Farrell'), ('Matthew Genting', 'Gerald Butters'), ('Anna Mackenzie', 'Darren Smith'), ('Joan Coplin', 'Timothy Baker'), ('Ramnaresh Sarwin', 'Florence Bader'), ('Douglas Jones', 'David Jones'), ('Henrietta Rumney', 'Matthew Genting'), ('Henry Worthington-Smyth', 'Tracy Smith'), ('Millicent Purview', 'Tracy Smith'), ('John Hunt', 'Millicent Purview'), ('Erica Crumpet', 'Tracy Smith')]


In [41]:
#Q12

c.execute("""SELECT Facilities.name as facility, SUM(CASE WHEN Bookings.memid = 0 THEN 0 ELSE Bookings.slots END) as usage
            FROM Bookings
            FULL OUTER JOIN Facilities
            ON Facilities.facid
            GROUP BY Facilities.name""")

print(c.fetchall())

[('Badminton Court', 6791), ('Massage Room 1', 6791), ('Massage Room 2', 6791), ('Pool Table', 6791), ('Snooker Table', 6791), ('Squash Court', 6791), ('Table Tennis', 6791), ('Tennis Court 1', None), ('Tennis Court 2', 6791)]


In [45]:
#Q13

c.execute("""SELECT Facilities.name as facility, 
            SUM(CASE WHEN Bookings.memid = 0 THEN 0 ELSE Bookings.slots END) as usage,
            strftime('%m', Bookings.starttime) as month,
            strftime('%y', Bookings.starttime) as year
            FROM Bookings
            FULL OUTER JOIN Facilities
            ON Facilities.facid
            GROUP BY Facilities.name, month,year""")

print(c.fetchall())


[('Badminton Court', 1061, '07', None), ('Badminton Court', 2531, '08', None), ('Badminton Court', 3199, '09', None), ('Massage Room 1', 1061, '07', None), ('Massage Room 1', 2531, '08', None), ('Massage Room 1', 3199, '09', None), ('Massage Room 2', 1061, '07', None), ('Massage Room 2', 2531, '08', None), ('Massage Room 2', 3199, '09', None), ('Pool Table', 1061, '07', None), ('Pool Table', 2531, '08', None), ('Pool Table', 3199, '09', None), ('Snooker Table', 1061, '07', None), ('Snooker Table', 2531, '08', None), ('Snooker Table', 3199, '09', None), ('Squash Court', 1061, '07', None), ('Squash Court', 2531, '08', None), ('Squash Court', 3199, '09', None), ('Table Tennis', 1061, '07', None), ('Table Tennis', 2531, '08', None), ('Table Tennis', 3199, '09', None), ('Tennis Court 1', None, None, None), ('Tennis Court 2', 1061, '07', None), ('Tennis Court 2', 2531, '08', None), ('Tennis Court 2', 3199, '09', None)]


In [46]:
conn.close()